In [1]:
# Standard Imports
from __future__ import print_function
import numpy as np
import pandas as pd
import os
import sys
from collections import defaultdict
from importlib import reload
from bs4 import BeautifulSoup
import requests
import scipy.stats as scs
import time
import seaborn as sns

import matplotlib.pyplot as plt

# Modeling
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import datasets, layers, models
# import keras.backend as K
# from tensorflow.keras.constraints import min_max_norm, non_neg
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

# Load MongoDB
from pymongo import MongoClient
client = MongoClient()
# Access/Initiate Database
db = client['producer_db']
# Access/Initiate Table
mfcc_tab = db['mfcc']
tab = db['songs']
collection = db.tab
mfcc_collection = db.mfcc_tab

# Authorize Spotify API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = os.environ['SPOTIFY_CLIENT_ID']
client_secret = os.environ['SPOTIFY_CLIENT_SECRET']
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Discogs token
discogs_token = os.environ['DISCOGS_TOKEN']

# Plotly
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import colorlover as cl
from IPython.display import HTML
plotly_username = os.environ['PLOTLY_USERNAME']
plotly_api_key = os.environ['PLOTLY_API_KEY']
plotly.tools.set_credentials_file(username=plotly_username, api_key=plotly_api_key)
#set colormap
Set3_10 = cl.scales['10']['qual']['Set3']
cm_10 = list(zip(np.linspace(0,1,10),Set3_10)) 

# Custom Modules
import src.knn_modeling as km

In [2]:
reload(km)
pv = km.ProductionValue(collection = mfcc_collection, sp = sp, discogs_token = discogs_token)
pv.fit_knn('mfcc_highpass')

In [32]:
a,b,c = pv.query('beat it', artist="michael jackson", use_spotify=True)
print()
print('Likely Producers:')
print(a)
print()
print('Actual Producer:')
print(c)
print()
print('Similar Songs')
b

beat it artist:michael jackson


TypeError: 'NoneType' object is not iterable

In [33]:
mfcc_collection.find_one()['basic_genre']

'country'

In [35]:
song_df = pv.song_df

In [43]:
song_df.head()

,track,artist,album,mfcc_highpass,genre,producer
0,Act Naturally,Buck Owens and the Buckaroos,The Best of Buck Owens,"[[-106.0409148180113, -114.71286150048788, -15...",country,George Martin
1,Alfie,Cilla Black,Cilla Black singles chronology,"[[-115.42859278964515, -133.7129072512278, -20...",pop,George Martin
2,Amber Cascades,America,Hideaway,"[[-132.7895650459967, -149.0103345463059, -205...",country,George Martin
3,Anyone Who Had a Heart,Dionne Warwick,Anyone Who Had a Heart,"[[-417.5704948136371, -400.7918664259179, -375...",soul,George Martin
4,Baby It's You,The Shirelles,Baby It's You,"[[-199.51104878028514, -215.14526944983157, -2...",soul,George Martin


In [39]:
genres = song_df['genre'].unique()

In [42]:
pv.y_labels

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0,